In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from scipy.signal import tukey
import pandas as pd
import clemb
from clemb.forward_model import forward_model, fullness, esol, rk4, es
import IPython.core.debugger
dbg = IPython.core.debugger.Pdb()


In [ ]:
def error(y1, y2, t1, t2):
    f1 = interp1d(t1, y1)
    err = y2 - f1(t2)
    return err

In [ ]:
if True:
    c = clemb.Clemb(clemb.LakeDataFITS(), clemb.WindDataCSV(), start='2018-02-01 ', end='2018-06-06')

In [ ]:
if True:
    rp = c.run([0,1])
    s = pd.HDFStore('clemb_output_forward_model.h5')
    s['df'] = rp
    s.close()

In [ ]:
s = pd.HDFStore('clemb_output_forward_model.h5')
df = s['df']
s.close()

In [ ]:
startdate = '2018-02-01'
enddate = '2018-06-06'
start = pd.Timestamp(startdate)
end = pd.Timestamp(enddate)

pwr = df.loc[0,'pwr']*0.0864
pwr = pwr.loc[start:end]
datetime = pwr.index

melt = df.loc[0,'fmelt']
melt = melt.loc[start:end]

orig_steam = df.loc[0, 'steam']
orig_steam = orig_steam.loc[start:end]

orig_mevap = df.loc[0, 'evfl']
orig_mevap = orig_mevap.loc[start:end]

ll = df.loc[0,'llvl']
ll = ll.loc[start:end]

t = df.loc[0,'t']
t = t.loc[start:end]

ndays = (datetime[-1] - datetime[0]).days
dt = 1.
nsteps = int(ndays/float(dt))
time = np.linspace(0,ndays,nsteps+1)
T0 = t.values[0]

y = np.zeros((nsteps+1, 3))
a,vol = fullness(ll.values[0])
density = 1.003 - 0.00033 * T0
M0 = vol*density
y[0] = [T0, M0, 0]
steams = np.zeros(nsteps+1)
mevaps = np.zeros(nsteps+1)
esols = np.zeros(nsteps+1)
for i in range(nsteps):
    a, vol = fullness(ll.values[i])
    solar = esol(i*dt, a, datetime)
    esols[i] = solar
    Mout = 0.
    enthalpy = 6.0
    windspeed = 4.5
    y_new, steam, mevap = forward_model(y[i], dt, a, vol, pwr.values[i], 
                                 melt.values[i], Mout, solar, enthalpy, windspeed)
    steams[i] = steam
    mevaps[i] = mevap
    y[i+1] = y_new
    
mpl.rcParams['figure.subplot.hspace'] = 0.5
plt.figure(figsize=(10,8))
ax1 = plt.subplot(4,2,1)
ax1.plot(time, pwr.values[:]/0.0864, ls='--')
ax1.set_title('Heat input rate')

ax2 = plt.subplot(4,2,3)
ax2.plot(time, melt.values[:], ls='--')
ax2.set_title('Meltwater inflow')

ax3 = plt.subplot(4,2,5)
ax3.plot(time, mevaps)
ax3.plot(time, orig_mevap, ls='--')
ax3.set_title('Evaporation')

ax4 = plt.subplot(4,2,2)
ax4.plot(time, y[:,0])
ax4.plot(time, t.values[:], ls='--')
ax4.set_title('Lake temperature')

ax5 = plt.subplot(4,2,4)
ax5.plot(time, y[:,0] - t.values[:])
ax5.set_title('Temperature difference')

ax6 = plt.subplot(4,2,6)
ax6.plot(time, steams)
ax6.plot(time, orig_steam, ls='--')
ax6.set_title('Steam')

ax7 = plt.subplot(4,2,7)
ax7.plot(time, y[:,1])
_, vol = fullness(ll.values[:])
rho = 1.003 - 0.00033 * t.values[:]
mass = vol*rho
ax7.plot(time, mass, ls='--')
ax7.set_title('Lake mass')

ax8 = plt.subplot(4,2,8)
ax8.plot(time, y[:,1]- mass)
ax8.set_title('Mass difference')


### Synthetic input

In [ ]:
# matplotlib.rcParams['axes.prop_cycle']
cl1 = '#1f77b4'
cl2 = '#ff7f0e'
cl3 = '#2ca02c'

In [ ]:
def outflow(level, area=0.2):
    """
    Compute outflow based on bernoulli equation.
    Returns outflow in km^3/day.
    """
    g = 9.81 # m/s^2
    h = level - 45.31
    if h <= 0.:
        return 0.
    v = np.sqrt(2*g*h)
    vol = area * v # ouflow volume in m^3/s
    vol *= 1e-3 * 86400 # convert to km^3/day
    return vol

In [ ]:
def volume(mass, temp):
    """
    Compute volume based on mass and temperature.
    Return volume in km^3
    """
    rho = 1.003 - 0.00033 * temp
    return mass/rho

In [ ]:
def mass(volume, temp):
    """
    Return mass based on volume and temperature.
    """
    rho = 1.003 - 0.00033 * temp
    return volume * rho

In [ ]:
def level(volume, lakearea):
    """
    Compute lake level based on volume and surface area.
    Return lake level in m.
    """
    return volume * 1e3 / lakearea

In [ ]:
f = 0.01
nsteps = 2001
tmax = 300.
t = np.linspace(0, tmax, nsteps)
dt = tmax/(nsteps-1)
qi_sinus = False
qi_tukey = True
if qi_sinus:
    qi = np.sin(2.*np.pi*f*t + np.deg2rad(15))*300.+300.
    qi *= 0.0864
if qi_tukey:
    qi = np.ones(nsteps)*400.*0.0864
    qi *= tukey(nsteps, 0.7)

datetime = pd.date_range('1/1/2017', periods=nsteps+1, freq='D')
y = np.zeros((nsteps, 3))
prm = np.zeros((nsteps-1, 4))
T = 25.
V = 8800
A = 194162
Mc = 10.
H = 6.0
WS = 4.5
X = 10.
M = mass(V, T)
y[0,:] = [T, M, X]
ll = level(V, A)
Mo = outflow(ll)


for i in range(nsteps-1):
    prm[i, 0] = Mc
    prm[i, 1] = Mo
    prm[i, 2] = ll
    solar = esol(t[i], A, datetime)
    y_new, steam, mevap = forward_model(y[i], dt, A, V, qi[i],
                                        Mc, Mo, solar, H, WS,
                                        method='rk4')
    prm[i, 3] = mevap
    V = volume(y_new[1], y_new[0])
    ll = level(V, A)
    Mo = outflow(ll)
    y[i+1, :] = y_new[:]
    
if True:
    mpl.rcParams['figure.subplot.hspace'] = 0.5
    fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(14, 10))
    
    axs[0,0].plot(t, qi/0.0864, ls='--')
    axs[0,0].set_title('Heat input rate')

    axs[0,1].plot(t[:-1], prm[:, 0])
    axs[0,1].set_title('Meltwater inflow')

    axs[1,0].plot(t, y[:,1])
    axs[1,0].set_title('Lake mass')

    axs[1,1].plot(t, y[:,0])
    axs[1,1].set_title('Lake temperature')
    
    axs[2,0].plot(t[:-1], prm[:,1])
    axs[2,0].set_title('Outflow')
    
    axs[2,1].plot(t[:-1], prm[:,2])
    axs[2,1].set_title('Lake level')
    
    axs[3,0].plot(t, y[:,2])
    axs[3,0].set_title('Mg++ concentration')

    axs[3,1].plot(t[:-1], prm[:, 3])
    axs[3,1].set_title('Evaporated steam')


    


### Equations from Hurst et al., 1991
Energy balance for the lake:

$$
\frac{d}{dt}Q = Q_i - Q_e - cTM_o + cT_sM_s
$$

$Q$ = total energy of the lake

$T$ = temperature at any time $t$

$c$ = specific heat of lake water

$Q_i$ = heat input at the base of the lake

$Q_e$ = heat loss due to evaporation, radiation, solar heating (gain)

$M_o$ = total rate of outflow

$M_s$ = inflow rate from melt

$T_s$ = temperature of inflow

Assuming $T_s = 0 ^{\circ}C$:

$$
\begin{aligned}
\frac{d}{dt}Q & = Q_i - Q_e - cTM_o \\
              & = \frac{d}{dt}[cMT] \\
              & = cM\frac{dT}{dt} + cT\frac{dM}{dt}
\end{aligned}
$$

$$
\Rightarrow \qquad \frac{dT}{dt} = \frac{1}{cM}\left(Q_i - Q_e - cTM_o -cT\frac{dM}{dt}\right) 
$$

$M$ = mass of the water in the lake at time t

Mass balance:

$$
\frac{dM}{dt} = M_i + M_s - M_o - M_e
$$

$M_i$ = rate at which water or steam is added through the volcanic vent

$M_e$ = rate of evaporation losses

Ion concentration balance:

$$
\begin{aligned}
\frac{d}{dt}[MZ] & = Z_i M_i - Z M_o\\
\frac{dM}{dt} + M_o & = \frac{1}{Z} ( Z_i M_i - M \frac{dZ}{dt} ) 
\end{aligned}
$$

$Z$ = ion concentration in the lake

$Z_iM_i$ = rate of addition of ions through the lake bottom

If $Z_iM_i = 0$:

$$
\frac{dM}{dt} = -M_o - \frac{M}{Z}\frac{dZ}{dt}
$$

### Equations from code

Lake mass:

$$
\begin{aligned}
\rho & = 1.003 - 0.00033T \\
V & = f(h)\\
M & = V * \rho
\end{aligned}
$$
$\rho$ = density of lake water

$V$ = lake volume

$h$ = lake level

Outflow:

$$
M_o = M_n - M_{n-1} + M_{n-1} \left (0.98\frac{M_{n-1}Z_{n-1}}{M_nZ_n} -1.0 \right )
$$

Long wave radiation loss:

$$
\dot{E}_{long} = 5.67e^{-8}A[0.97(T - 1 + 273.15)^4 -  0.8(0.9 + 273.15)^4]
$$

$A$ = lake surface area

Free and forced convection:

$$
\begin{aligned}
\dot{E}_{evap} & = \\
&  A \sqrt{\left [ 2.2 (T - 1 - 0.9)^{\frac{1}{3}} (6.112e^{\frac{17.62(T-1)}{243.12(T-1)}} - 6.5) \right ] ^2+ \left [ \left ( 4.07e^{-3} \frac{w^{0.8}}{500^{0.2}} - \frac{1.107e^{-2}}{500} \right ) \left ( \frac{1}{800} ( 6.112e^{\frac{17.62(T-1)}{243.12(T-1)}} - 6.5) \right ) 2400000 \right ]^2}
\end{aligned}
$$

$w$ = wind speed

Solar heating:

$$
\dot{E}_{solar} = \Delta t A 1.5e^{-5} \left [ 1 + 0.5 cos \left (\frac{((m-1)3.14)}{6.0} \right ) \right ]
$$

$m$ = month of the year

Heat loss due to evaporation radiation and solar heating 

$$
Q_e = \dot{E}_{long} + \dot{E}_{evap} \left ( 1+0.948 \frac{1005}{2.4e^6} \frac{T - 1 - 0.9}{6.335e^{-3} + 6.718e^{-4}(T-1) -2.0887e^{-5}(T-1)^2 + 7.3095e^{-7}(T-1)^3 - 2.2e^{-3}} \right ) - \dot{E}_{solar}
$$

$$
M_e = \dot{E}_{evap}2.4e^{-12}
$$